In [1]:
import xarray as xr
import pandas as pd
import numpy as np

In [2]:
from open_experiment import control_land, fixedSM_land, control_deaccu, fixedSM_deaccu, control_sfc, fixedSM_sfc

In [3]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [4]:
sum_control_land = control_land.isel(depth=slice(0,5)).sum('depth').load()
sum_fixedSM_land = fixedSM_land.isel(depth=slice(0,5)).sum('depth').load()

In [5]:
print(len(sum_control_land.time),len(sum_fixedSM_land.time))

1434 1434


In [4]:
path='/scratch/wcq7pz/exp_levante_post/'

topo5km = xr.open_dataset(path+'topography_dom03_5km.nc')
topo5km.coords['lon'] = (topo5km.coords['lon'] + 180) % 360 - 180
topo5km  = topo5km.sortby(topo5km.lon)

frland=xr.open_dataset(path+'fr_land_dom03_5km.nc')
frland.coords['lon'] = (frland.coords['lon'] + 180) % 360 - 180
frland = frland.sortby(frland.lon)

In [5]:
# Repeat the variable along the time dimension of datasets
ds_topo = topo5km['topography_c'].broadcast_like(control_sfc['time']).load()
ds_landf = frland['fr_land'].broadcast_like(control_sfc['time']).load()

In [8]:
# Remove high topography and keep land surface (i.e. omit rivers)
ncontrol_land = sum_control_land.where((ds_topo<=500)&(ds_landf==1),np.nan).load()
nfixedSM_land = sum_fixedSM_land.where((ds_topo<=500)&(ds_landf==1),np.nan).load()

In [6]:
ncontrol_tqv = control_sfc.tqv_dia.where((ds_topo<=500)&(ds_landf==1),np.nan).load()
nfixedSM_tqv = fixedSM_sfc.tqv_dia.where((ds_topo<=500)&(ds_landf==1),np.nan).load()

#### Load objects and dataframes with stage information

In [7]:
### read Stage info
df_ocs_control_AB = pd.read_pickle('pkl_files/df_ocs_control_AB.pkl'); 
df_ocs_fixedSM_AB = pd.read_pickle('pkl_files/df_ocs_fixedSM_AB.pkl'); 
df_ocs_control_SESA = pd.read_pickle('pkl_files/df_ocs_control_SESA.pkl'); 
df_ocs_fixedSM_SESA = pd.read_pickle('pkl_files/df_ocs_fixedSM_SESA.pkl'); 


#### Define functions

In [8]:
def mirrored(maxval, inc=1):
    x = np.arange(inc, maxval, inc)
    if x[-1] != maxval:
        x = np.r_[x, maxval]
    return np.r_[-x[::-1], 0, x]


In [9]:
def concat_spatial_composites(var, df,  dist= 1,grid = 0.045,timelag=False,dt='1hours'):
    
    time_cord = np.arange(len(df));
    
    if timelag==True:
        comp = xr.concat([var.sel(time=df.time.values[i]-pd.Timedelta(dt)).sel(lon=slice(df.longitude.values[i]-dist,df.longitude.values[i]+dist),
                                  lat=slice(df.latitude.values[i]-dist,df.latitude.values[i]+dist)).update(
    {"lon": ("lon", 100*(mirrored((dist-grid), grid))),"lat": ("lat", 100*(mirrored((dist-grid), grid))),
     "time": ("time",[time_cord[i]])}) for i in range(len(df)) ],'time')
    else:
        comp = xr.concat([var.sel(time=df.time.values[i]).sel(lon=slice(df.longitude.values[i]-dist,df.longitude.values[i]+dist),
                                  lat=slice(df.latitude.values[i]-dist,df.latitude.values[i]+dist)).update(
    {"lon": ("lon", 100*(mirrored((dist-grid), grid))),"lat": ("lat", 100*(mirrored((dist-grid), grid))),
     "time": ("time",[time_cord[i]])}) for i in range(len(df)) ],'time');
    
    
    
    return comp

In [14]:
def sel_composite(df,var1,namevar='w_so',dist=2,timelag=False,dt=1,varmean='mean_SM_env',varstd='stad_SM_env'):
    """
    df = dataframe
    var1, var2 = 3D dataset of dimensions time,lat,lon for precipitation and SM, respectively.
    """
    if timelag==True:
        
        df_cat = pd.concat([pd.DataFrame([{'time':df.time.values[i]-pd.Timedelta(dt),'local_time':df.local_time.values[i]-pd.Timedelta(dt),
                                           'feature':df.feature.values[i],
        varmean:var1.sel(time=df.time.values[i]-pd.Timedelta(dt)).sel(lon=slice(
        df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
        df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).mean(['lat','lon'])[namevar].values, 
        varstd:var1.sel(time=df.time.values[i]-pd.Timedelta(dt)).sel(lon=slice(
        df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
        df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).std(['lat','lon'])[namevar].values}]) for i in range(len(df))],
ignore_index=False)
    else:
        df_cat = pd.concat([pd.DataFrame([{'time':df.time.values[i],'local_time':df.local_time.values[i],'feature':df.feature.values[i],
        varmean:var1.sel(time=df.time.values[i]).sel(lon=slice(
        df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
        df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).mean(['lat','lon'])[namevar].values, 
        varstd:var1.sel(time=df.time.values[i]).sel(lon=slice(
        df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
        df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).std(['lat','lon'])[namevar].values}]) for i in range(len(df))],
ignore_index=False) 
       
    df_cat = df_cat.reset_index(drop=True);
    df_cat[varmean] =df_cat[varmean].astype(float)
    df_cat[varstd] =df_cat[varstd].astype(float)
    return df_cat

#### Compute the mean and std of SM in the near environment. (Environment definition: Select around "dist" times the radius of the systems)

In [36]:
##### Select around "dist" times the radius of the object
### Amazon
sm_env_all_c_2r = sel_composite(df_ocs_control_AB[(
    df_ocs_control_AB.longitude>-76.5)&(df_ocs_control_AB.longitude<-55.5)].reset_index(),
                               ncontrol_land,dist=2)
sm_env_all_c_2r.to_pickle('pkl_files/sm_env_all_c_2r_notoprivr.pkl')


sm_env_all_f_2r = sel_composite(df_ocs_fixedSM_AB[(
    df_ocs_fixedSM_AB.longitude>-76.5)&(df_ocs_fixedSM_AB.longitude<-55.5)].reset_index(),
                               nfixedSM_land,dist=2)
sm_env_all_f_2r.to_pickle('pkl_files/sm_env_all_f_2r_notoprivr.pkl')

### SESA
sm_env_all_c_2r_SESA =sel_composite(df_ocs_control_SESA[(
   df_ocs_control_SESA.longitude>-76.5)&(df_ocs_control_SESA.latitude>-24.5)].reset_index(),
                               ncontrol_land,dist=2)
sm_env_all_c_2r_SESA.to_pickle('pkl_files/sm_env_all_c_SESA_2r_notoprivr.pkl')


sm_env_all_f_2r_SESA = sel_composite(df_ocs_fixedSM_SESA[(
    df_ocs_fixedSM_SESA.longitude>-76.5)&(df_ocs_fixedSM_SESA.latitude>-24.5)].reset_index(),
                               nfixedSM_land,dist=2)
sm_env_all_f_2r_SESA.to_pickle('pkl_files/sm_env_all_f_SESA_2r_notoprivr.pkl')

In [18]:
### other sfc variables
### Amazon
tqv_env_all_c_2r = sel_composite(df_ocs_control_AB[(
    df_ocs_control_AB.longitude>-76.5)&(df_ocs_control_AB.longitude<-55.5)].reset_index(),
                               ncontrol_tqv.to_dataset(name='tqv_dia'),dist=2,namevar='tqv_dia',varmean='mean_TQV_env',varstd='std_TQV_env')
tqv_env_all_c_2r.to_pickle('pkl_files/tqv_env_all_c_2r_notoprivr.pkl')

tqv_env_all_f_2r = sel_composite(df_ocs_fixedSM_AB[(
    df_ocs_fixedSM_AB.longitude>-76.5)&(df_ocs_fixedSM_AB.longitude<-55.5)].reset_index(),
                               nfixedSM_tqv.to_dataset(name='tqv_dia'),dist=2,namevar='tqv_dia',varmean='mean_TQV_env',varstd='std_TQV_env')
tqv_env_all_f_2r.to_pickle('pkl_files/tqv_env_all_f_2r_notoprivr.pkl')

### SESA
tqv_env_all_c_2r_SESA = sel_composite(df_ocs_control_SESA[(
    df_ocs_control_SESA.longitude>-76.5)&(df_ocs_control_SESA.latitude>-24.5)].reset_index(),
                               ncontrol_tqv.to_dataset(name='tqv_dia'),dist=2,namevar='tqv_dia',varmean='mean_TQV_env',varstd='std_TQV_env')
tqv_env_all_c_2r_SESA.to_pickle('pkl_files/tqv_env_all_c_SESA_2r_notoprivr.pkl')

tqv_env_all_f_2r_SESA = sel_composite(df_ocs_fixedSM_SESA[(
    df_ocs_fixedSM_SESA.longitude>-76.5)&(df_ocs_fixedSM_SESA.latitude>-24.5)].reset_index(),
                               nfixedSM_tqv.to_dataset(name='tqv_dia'),dist=2,namevar='tqv_dia',varmean='mean_TQV_env',varstd='std_TQV_env')
tqv_env_all_f_2r_SESA.to_pickle('pkl_files/tqv_env_all_f_SESA_2r_notoprivr.pkl')

### Make spatial composites of detected objects 

In [24]:
sm_ti_c = concat_spatial_composites(ncontrol_land,df_ocs_control_AB[(
    df_ocs_control_AB.stage=='t_i')&(df_ocs_control_AB.longitude>-76.5)&(df_ocs_control_AB.longitude<-55.5)],dist=5)
sm_ti_f = concat_spatial_composites(nfixedSM_land,df_ocs_fixedSM_AB[(
    df_ocs_fixedSM_AB.stage=='t_i')&(df_ocs_fixedSM_AB.longitude>-76.5)&(df_ocs_fixedSM_AB.longitude<-55.5)],dist=5)

sm_ti_c.to_netcdf('spatial_composites/sm_ti_c_5deg_notoprivr.nc'); 
sm_ti_f.to_netcdf('spatial_composites/sm_ti_f_5deg_notoprivr.nc'); 

In [27]:
deaccu_ti_c = concat_spatial_composites(control_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_ocs_control_AB[(
    df_ocs_control_AB.stage=='t_i')&(df_ocs_control_AB.longitude>-76.5)&(df_ocs_control_AB.longitude<-55.5)],dist=5)
deaccu_ti_f = concat_spatial_composites(fixedSM_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_ocs_fixedSM_AB[(
    df_ocs_fixedSM_AB.stage=='t_i')&(df_ocs_fixedSM_AB.longitude>-76.5)&(df_ocs_fixedSM_AB.longitude<-55.5)],dist=5)

deaccu_ti_c.to_netcdf('spatial_composites/deaccu_ti_c_5deg.nc'); 
deaccu_ti_f.to_netcdf('spatial_composites/deaccu_ti_f_5deg.nc'); 

In [30]:
sfc_ti_c = concat_spatial_composites(control_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_ocs_control_AB[(
     df_ocs_control_AB.stage=='t_i')&(df_ocs_control_AB.longitude>-76.5)&(df_ocs_control_AB.longitude<-55.5)],dist=5)
sfc_ti_f = concat_spatial_composites(fixedSM_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_ocs_fixedSM_AB[(
     df_ocs_fixedSM_AB.stage=='t_i')&(df_ocs_fixedSM_AB.longitude>-76.5)&(df_ocs_fixedSM_AB.longitude<-55.5)],dist=5)

sfc_ti_c.to_netcdf('spatial_composites/sfc_ti_c_notopnorivr_5deg.nc'); 
sfc_ti_f.to_netcdf('spatial_composites/sfc_ti_f_notopnorivr_5deg.nc'); 

##### Make spatial composites with time lags

##### 3 hour before

In [14]:
df_stage_c_3hb = df_ocs_control_AB[(df_ocs_control_AB.time!='2018-03-31 02:00:00')&(df_ocs_control_AB.time!='2018-03-31 03:00:00')&(
                df_ocs_control_AB.time!='2018-03-31 04:00:00')&(df_ocs_control_AB.time!='2017-03-31 03:00:00')&(df_ocs_control_AB.time!='2017-03-31 04:00:00')]
df_stage_f_3hb = df_ocs_fixedSM_AB[(df_ocs_fixedSM_AB.time!='2018-03-31 02:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 03:00:00')&(
                df_ocs_fixedSM_AB.time!='2018-03-31 04:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 03:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 04:00:00')]


In [15]:
sm_ti_c_3hb = concat_spatial_composites(ncontrol_land,df_stage_c_3hb[(
    df_stage_c_3hb.stage=='t_i')&(df_stage_c_3hb.longitude>-76.5)&(df_stage_c_3hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='3hours')
sm_ti_f_3hb = concat_spatial_composites(nfixedSM_land,df_stage_f_3hb[(
    df_stage_f_3hb.stage=='t_i')&(df_stage_f_3hb.longitude>-76.5)&(df_stage_f_3hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='3hours')


In [16]:
sm_ti_c_3hb.to_netcdf('spatial_composites/sm_ti_c_3hb_5reg_AB_notoprivr.nc'); 
sm_ti_f_3hb.to_netcdf('spatial_composites/sm_ti_f_3hb_5reg_AB_notoprivr.nc'); 

In [18]:
deaccu_ti_c_3hb = concat_spatial_composites(control_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_c_3hb[(
    df_stage_c_3hb.stage=='t_i')&(df_stage_c_3hb.longitude>-76.5)&(df_stage_c_3hb.longitude<-55.5)],dist=5,timelag=True,dt='3hours')
deaccu_ti_f_3hb = concat_spatial_composites(fixedSM_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_f_3hb[(
    df_stage_f_3hb.stage=='t_i')&(df_stage_f_3hb.longitude>-76.5)&(df_stage_f_3hb.longitude<-55.5)],dist=5,timelag=True,dt='3hours')

In [19]:
deaccu_ti_c_3hb.to_netcdf('spatial_composites/deaccu_ti_3hb_c_5deg_AB.nc'); 
deaccu_ti_f_3hb.to_netcdf('spatial_composites/deaccu_ti_3hb_f_5deg_AB.nc'); 

In [20]:
sfc_ti_c_3hb = concat_spatial_composites(control_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_c_3hb[(
     df_stage_c_3hb.stage=='t_i')&(df_stage_c_3hb.longitude>-76.5)&(df_stage_c_3hb.longitude<-55.5)],dist=5,timelag=True,dt='3hours')
sfc_ti_f_3hb = concat_spatial_composites(fixedSM_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_f_3hb[(
     df_stage_f_3hb.stage=='t_i')&(df_stage_f_3hb.longitude>-76.5)&(df_stage_f_3hb.longitude<-55.5)],dist=5,timelag=True,dt='3hours')

In [21]:
sfc_ti_c_3hb.to_netcdf('spatial_composites/sfc_ti_3hb_c_5deg_AB.nc'); 
sfc_ti_f_3hb.to_netcdf('spatial_composites/sfc_ti_3hb_f_5deg_AB.nc'); 

##### 6 hours before

In [22]:
df_stage_c_6hb = df_stage_c_3hb[(df_stage_c_3hb.time!='2018-03-31 05:00:00')&(df_stage_c_3hb.time!='2018-03-31 06:00:00')&(df_stage_c_3hb.time!='2018-03-31 07:00:00')&(
    df_stage_c_3hb.time!='2017-03-31 05:00:00')&(df_stage_c_3hb.time!='2017-03-31 06:00:00')&(df_stage_c_3hb.time!='2017-03-31 07:00:00')&(df_stage_c_3hb.time!='2019-03-31 05:00:00')&(
    df_stage_c_3hb.time!='2019-03-31 06:00:00')&(df_stage_c_3hb.time!='2019-03-31 07:00:00')]
df_stage_f_6hb = df_stage_f_3hb[(df_stage_f_3hb.time!='2018-03-31 05:00:00')&(df_stage_f_3hb.time!='2018-03-31 06:00:00')&(df_stage_f_3hb.time!='2018-03-31 07:00:00')&(
    df_stage_f_3hb.time!='2017-03-31 05:00:00')&(df_stage_f_3hb.time!='2017-03-31 06:00:00')&(df_stage_f_3hb.time!='2017-03-31 07:00:00')&(
    df_stage_f_3hb.time!='2019-03-31 05:00:00')&(df_stage_f_3hb.time!='2019-03-31 06:00:00')&(df_stage_f_3hb.time!='2019-03-31 07:00:00')]


In [24]:
sm_ti_c_6hb = concat_spatial_composites(ncontrol_land,df_stage_c_6hb[(
    df_stage_c_6hb.stage=='t_i')&(df_stage_c_6hb.longitude>-76.5)&(df_stage_c_6hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='6hours')
sm_ti_f_6hb = concat_spatial_composites(nfixedSM_land,df_stage_f_6hb[(
    df_stage_f_6hb.stage=='t_i')&(df_stage_f_6hb.longitude>-76.5)&(df_stage_f_6hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='6hours')

In [25]:
sm_ti_c_6hb.to_netcdf('spatial_composites/sm_ti_c_6hb_5reg_notoprivr.nc'); 
sm_ti_f_6hb.to_netcdf('spatial_composites/sm_ti_f_6hb_5reg_notoprivr.nc'); 

In [28]:
deaccu_ti_c_6hb = concat_spatial_composites(control_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_c_6hb[(
    df_stage_c_6hb.stage=='t_i')&(df_stage_c_6hb.longitude>-76.5)&(df_stage_c_6hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='6hours')
deaccu_ti_f_6hb = concat_spatial_composites(fixedSM_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_f_6hb[(
    df_stage_f_6hb.stage=='t_i')&(df_stage_f_6hb.longitude>-76.5)&(df_stage_f_6hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='6hours')

In [29]:
np.shape(deaccu_ti_f_6hb.tot_prec)

(1847, 223, 223)

In [39]:
np.shape(deaccu_ti_f_6hb.tot_prec)

(1847, 223, 223)

In [30]:
deaccu_ti_c_6hb.to_netcdf('spatial_composites/deaccu_ti_c_6hb_5reg_AB.nc'); 
deaccu_ti_f_6hb.to_netcdf('spatial_composites/deaccu_ti_f_6hb_5reg_AB.nc'); 

In [31]:
sfc_ti_c_6hb = concat_spatial_composites(control_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_c_6hb[(
     df_stage_c_6hb.stage=='t_i')&(df_stage_c_6hb.longitude>-76.5)&(df_stage_c_6hb.longitude<-55.5)],dist=5,timelag=True,dt='6hours')
sfc_ti_f_6hb = concat_spatial_composites(fixedSM_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_f_6hb[(
     df_stage_f_6hb.stage=='t_i')&(df_stage_f_6hb.longitude>-76.5)&(df_stage_f_6hb.longitude<-55.5)],dist=5,timelag=True,dt='6hours')

In [32]:
sfc_ti_c_6hb.to_netcdf('spatial_composites/sfc_ti_6hb_c_5deg_AB.nc'); 
sfc_ti_f_6hb.to_netcdf('spatial_composites/sfc_ti_6hb_f_5deg_AB.nc'); 

##### 12 hours before 

In [18]:
df_stage_c_12hb = df_ocs_control_AB[(df_ocs_control_AB.time!='2017-03-31 03:00:00')&(df_ocs_control_AB.time!='2017-03-31 04:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 05:00:00')&(df_ocs_control_AB.time!='2017-03-31 06:00:00')&(df_ocs_control_AB.time!='2017-03-31 07:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 08:00:00')&(df_ocs_control_AB.time!='2017-03-31 09:00:00')&(df_ocs_control_AB.time!='2017-03-31 10:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 11:00:00')&(df_ocs_control_AB.time!='2017-03-31 12:00:00')&(df_ocs_control_AB.time!='2017-03-31 13:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 02:00:00')&(df_ocs_control_AB.time!='2018-03-31 03:00:00')&(df_ocs_control_AB.time!='2018-03-31 04:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 05:00:00')&(df_ocs_control_AB.time!='2018-03-31 06:00:00')&(df_ocs_control_AB.time!='2018-03-31 07:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 08:00:00')&(df_ocs_control_AB.time!='2018-03-31 09:00:00')&(df_ocs_control_AB.time!='2018-03-31 10:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 11:00:00')&(df_ocs_control_AB.time!='2018-03-31 12:00:00')&(df_ocs_control_AB.time!='2018-03-31 13:00:00')&(
    df_ocs_control_AB.time!='2019-03-31 05:00:00')&(df_ocs_control_AB.time!='2019-03-31 06:00:00')&(df_ocs_control_AB.time!='2019-03-31 08:00:00')&(
    df_ocs_control_AB.time!='2019-03-31 09:00:00')&(df_ocs_control_AB.time!='2019-03-31 10:00:00')&(df_ocs_control_AB.time!='2019-03-31 12:00:00')&(
    df_ocs_control_AB.time!='2019-03-31 13:00:00')]

sm_ti_c_12hb = concat_spatial_composites(ncontrol_land,df_stage_c_12hb[(
    df_stage_c_12hb.stage=='t_i')&(df_stage_c_12hb.longitude>-76.5)&(df_stage_c_12hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='12hours')

In [19]:
sm_ti_c_12hb.to_netcdf('spatial_composites/sm_ti_c_12hb_5reg_notoprivr.nc'); 

In [36]:
deaccu_ti_c_12hb = concat_spatial_composites(control_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_c_12hb[(
    df_stage_c_12hb.stage=='t_i')&(df_stage_c_12hb.longitude>-76.5)&(df_stage_c_12hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='12hours')

In [37]:
deaccu_ti_c_12hb.to_netcdf('spatial_composites/deaccu_ti_c_12hb_5reg_notoprivr.nc'); 

In [38]:
sfc_ti_c_12hb = concat_spatial_composites(control_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_c_12hb[(
     df_stage_c_12hb.stage=='t_i')&(df_stage_c_12hb.longitude>-76.5)&(df_stage_c_12hb.longitude<-55.5)],dist=5,timelag=True,dt='12hours')
sfc_ti_c_12hb.to_netcdf('spatial_composites/sfc_ti_c_12hb_5reg_notoprivr.nc'); 

In [20]:
df_stage_f_12hb = df_ocs_fixedSM_AB[(df_ocs_fixedSM_AB.time!='2017-03-31 03:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 04:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 05:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 06:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 07:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 08:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 09:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 10:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 12:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 13:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 02:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 03:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 04:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 05:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 06:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 07:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 08:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 09:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 10:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 11:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 12:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 13:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 05:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 06:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 07:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 08:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 09:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 10:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 11:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 12:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 13:00:00')]

sm_ti_f_12hb = concat_spatial_composites(nfixedSM_land,df_stage_f_12hb[(
    df_stage_f_12hb.stage=='t_i')&(df_stage_f_12hb.longitude>-76.5)&(df_stage_f_12hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='12hours')

In [21]:
sm_ti_f_12hb.to_netcdf('spatial_composites/sm_ti_f_12hb_5reg_notoprivr.nc'); 

In [43]:
deaccu_ti_f_12hb = concat_spatial_composites(fixedSM_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_f_12hb[(
    df_stage_f_12hb.stage=='t_i')&(df_stage_f_12hb.longitude>-76.5)&(df_stage_f_12hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='12hours')

In [44]:
deaccu_ti_f_12hb.to_netcdf('spatial_composites/deaccu_ti_f_12hb_5reg_notoprivr.nc'); 

In [35]:
# del(sfc_ti_f_12hb)
sfc_ti_f_12hb = concat_spatial_composites(fixedSM_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_f_12hb[(
     df_stage_f_12hb.stage=='t_i')&(df_stage_f_12hb.longitude>-76.5)&(df_stage_f_12hb.longitude<-55.5)],
                                          dist=5,timelag=True,dt='12hours')

In [38]:
sfc_ti_f_12hb.to_netcdf('spatial_composites/sfc_ti_f_12hb_5reg_notoprivr.nc'); 

#### 24hb

In [49]:
df_stage_c_24hb = df_ocs_control_AB[(df_ocs_control_AB.time!='2017-03-31 03:00:00')&(df_ocs_control_AB.time!='2017-03-31 04:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 05:00:00')&(df_ocs_control_AB.time!='2017-03-31 06:00:00')&(df_ocs_control_AB.time!='2017-03-31 07:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 08:00:00')&(df_ocs_control_AB.time!='2017-03-31 09:00:00')&(df_ocs_control_AB.time!='2017-03-31 10:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 11:00:00')&(df_ocs_control_AB.time!='2017-03-31 12:00:00')&(df_ocs_control_AB.time!='2017-03-31 13:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 14:00:00')&(df_ocs_control_AB.time!='2017-03-31 15:00:00')&(df_ocs_control_AB.time!='2017-03-31 16:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 17:00:00')&(df_ocs_control_AB.time!='2017-03-31 18:00:00')&(df_ocs_control_AB.time!='2017-03-31 19:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 20:00:00')&(df_ocs_control_AB.time!='2017-03-31 21:00:00')&(df_ocs_control_AB.time!='2017-03-31 22:00:00')&(
    df_ocs_control_AB.time!='2017-03-31 23:00:00')&(df_ocs_control_AB.time!='2017-04-01 00:00:00')&(df_ocs_control_AB.time!='2017-04-01 01:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 02:00:00')&(df_ocs_control_AB.time!='2018-03-31 03:00:00')&(df_ocs_control_AB.time!='2018-03-31 04:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 05:00:00')&(df_ocs_control_AB.time!='2018-03-31 06:00:00')&(df_ocs_control_AB.time!='2018-03-31 07:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 08:00:00')&(df_ocs_control_AB.time!='2018-03-31 09:00:00')&(df_ocs_control_AB.time!='2018-03-31 10:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 11:00:00')&(df_ocs_control_AB.time!='2018-03-31 12:00:00')&(df_ocs_control_AB.time!='2018-03-31 13:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 14:00:00')&(df_ocs_control_AB.time!='2018-03-31 15:00:00')&(df_ocs_control_AB.time!='2018-03-31 16:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 17:00:00')&(df_ocs_control_AB.time!='2018-03-31 18:00:00')&(df_ocs_control_AB.time!='2018-03-31 19:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 20:00:00')&(df_ocs_control_AB.time!='2018-03-31 21:00:00')&(df_ocs_control_AB.time!='2018-03-31 22:00:00')&(
    df_ocs_control_AB.time!='2018-03-31 23:00:00')&(df_ocs_control_AB.time!='2018-04-01 00:00:00')&(df_ocs_control_AB.time!='2018-04-01 01:00:00')&(
    df_ocs_control_AB.time!='2018-04-01 02:00:00')&(df_ocs_control_AB.time!='2018-04-01 03:00:00')&(df_ocs_control_AB.time!='2018-04-01 04:00:00')&(
    df_ocs_control_AB.time!='2019-03-31 05:00:00')&(df_ocs_control_AB.time!='2019-03-31 06:00:00')&(df_ocs_control_AB.time!='2019-03-31 08:00:00')&(
    df_ocs_control_AB.time!='2019-03-31 09:00:00')&(df_ocs_control_AB.time!='2019-03-31 10:00:00')&(df_ocs_control_AB.time!='2019-03-31 12:00:00')&(
    df_ocs_control_AB.time!='2019-03-31 13:00:00')&(df_ocs_control_AB.time!='2019-03-31 16:00:00')&(df_ocs_control_AB.time!='2019-03-31 17:00:00')&(
    df_ocs_control_AB.time!='2019-03-31 18:00:00')&(df_ocs_control_AB.time!='2019-03-31 19:00:00')&(df_ocs_control_AB.time!='2019-04-01 01:00:00')]

sm_ti_c_24hb = concat_spatial_composites(ncontrol_land,df_stage_c_24hb[(
    df_stage_c_24hb.stage=='t_i')&(df_stage_c_24hb.longitude>-76.5)&(df_stage_c_24hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='24hours')

In [50]:
sm_ti_c_24hb.to_netcdf('spatial_composites/sm_ti_c_24hb_5reg_notoprivr.nc'); 

In [52]:
deaccu_ti_c_24hb = concat_spatial_composites(control_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_c_24hb[(
    df_stage_c_24hb.stage=='t_i')&(df_stage_c_24hb.longitude>-76.5)&(df_stage_c_24hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='24hours')

In [53]:
deaccu_ti_c_24hb.to_netcdf('spatial_composites/deaccu_ti_c_24hb_5reg_notoprivr.nc'); 

In [56]:
sfc_ti_c_24hb = concat_spatial_composites(control_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_c_24hb[(
     df_stage_c_24hb.stage=='t_i')&(df_stage_c_24hb.longitude>-76.5)&(df_stage_c_24hb.longitude<-55.5)],dist=5,
                                          timelag=True,dt='24hours')


In [57]:
sfc_ti_c_24hb.to_netcdf('spatial_composites/sfc_ti_c_24hb_5reg_notoprivr.nc'); 

In [58]:
df_stage_f_24hb = df_ocs_fixedSM_AB[(df_ocs_fixedSM_AB.time!='2017-03-31 03:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 04:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 05:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 06:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 07:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 08:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 09:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 10:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 12:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 13:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 14:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 15:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 16:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 17:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 18:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 19:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 20:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-03-31 21:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 22:00:00')&(df_ocs_fixedSM_AB.time!='2017-03-31 23:00:00')&(
    df_ocs_fixedSM_AB.time!='2017-04-01 01:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 02:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 03:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 04:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 05:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 06:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 07:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 08:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 09:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 10:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 11:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 12:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 13:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 14:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 15:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 16:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 17:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 18:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 19:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 20:00:00')&(df_ocs_fixedSM_AB.time!='2018-03-31 22:00:00')&(
    df_ocs_fixedSM_AB.time!='2018-03-31 23:00:00')&(df_ocs_fixedSM_AB.time!='2018-04-01 00:00:00')&(df_ocs_fixedSM_AB.time!='2018-04-01 01:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 05:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 06:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 07:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 08:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 09:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 10:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 11:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 12:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 13:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 14:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 15:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 16:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 17:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 18:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 19:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 20:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 21:00:00')&(df_ocs_fixedSM_AB.time!='2019-03-31 22:00:00')&(
    df_ocs_fixedSM_AB.time!='2019-03-31 23:00:00')]

sm_ti_f_24hb = concat_spatial_composites(nfixedSM_land,df_stage_f_24hb[(
    df_stage_f_24hb.stage=='t_i')&(df_stage_f_24hb.longitude>-76.5)&(df_stage_f_24hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='24hours')

In [59]:
sm_ti_f_24hb.to_netcdf('spatial_composites/sm_ti_f_24hb_5reg_notoprivr.nc'); 

In [60]:
deaccu_ti_f_24hb = concat_spatial_composites(fixedSM_deaccu[['tot_prec','acclhfl_s','accshfl_s']],df_stage_f_24hb[(
    df_stage_f_24hb.stage=='t_i')&(df_stage_f_24hb.longitude>-76.5)&(df_stage_f_24hb.longitude<-55.5)],
                                        dist=5,timelag=True,dt='24hours')

In [61]:
deaccu_ti_f_24hb.to_netcdf('spatial_composites/deaccu_ti_f_24hb_5reg_notoprivr.nc'); 

In [62]:
sfc_ti_f_24hb = concat_spatial_composites(fixedSM_sfc.where((ds_topo<=500)&(ds_landf==1),np.nan),df_stage_f_24hb[(
     df_stage_f_24hb.stage=='t_i')&(df_stage_f_24hb.longitude>-76.5)&(df_stage_f_24hb.longitude<-55.5)],
                                          dist=5,timelag=True,dt='24hours')

In [63]:
sfc_ti_f_24hb.to_netcdf('spatial_composites/sfc_ti_f_24hb_5reg_notoprivr.nc'); 